# 11 Object relational mapping

* Follow up on last exercise

* The issue to be solved
* Mapping strategies
* Specifying the mapping
* Object caching and lazy load
* Transactions
* Querying 
    LINQ
* Code generation

# Mapping strategies
### Simple values
More or less the same in programming language and SQL (numbers, booleans and string)

* Annoying difference: Dates

### Serious differences
* Lists and non-sql values
* Many to many relationships
* (Inheritance)
* Roles

# Lists and non-sql value
```java
class Road {
    String name;
    PolyLine location;
}
class PolyLine {
    List<Coordinate> points;
    boolean closed;
}
```
### How to represent PolyLine in the road table?

# convert to/from string
```java
class PolyLine {
    String toString(){
        return String.join(",", points.map(p-> ""+p.x+" "+p.y).toArray())+";"+closed;
    }
    static Road fromString(String str){
        String[] pline = str.split(";");
        closed = Boolean.valueOf(pline[1]);
        var coords = pline.split(","); # Java 10 has var - yay!
        points = cords.map(p -> new Coordinate(p)).collect(Collectors.toList());
    }    
}
```

# serialize, json,...
There is direct support for converting objects to/from strings in a number of ways:

* json libraries
* java's serializable (matches sql blobs quite well)
* xml libraries




# Null
Databases allow null in an integer column. Programming languages mostly do not.

* Default value (0, -1, MININT, MAXINT)
* In Java: Integer vs. int

# Mapping relationships

There are three fundamental cardinalities to consider:
* 1-1	Wife and Husband (most cultures)
* 1-M	Mother and children
* M-M	Cousins and Cousins

1-1 is references vs foreign keys (and of the tables can hold the foreign key)<br>
1-M is collections vs foreign keys (many side holds the foreign key)<br>
M-M is collections - database need a separate table.<br>


# Traversals
Order-Orderlines-Products

### Clases - you can have one-way 
For example - go from orderline to Product, but not the other way

### Relational model
You always have two way (by query, foreign key and index)

# Dealing with inheritance
Consider the inheritance structure (Vehicle is an abstract class)

<img src="images/inheritance1.png" width="40%"/>

## One big table
Using one big table, we need a type attribute, and then all fields from the entire hierarchy.
<img src="images/inheritance2.png" width="40%"/>

### Advantages: 
* Easy to work with.
* Polymorphism (type column)

### Disadvantage:
Null values

## Table per class
<img src="images/inheritance3.png" width="40%"/>

### Advantages:
* Easy – one to one
* Polymorphism
* Space efficient

### Disadvantages:
* More complicated to assemble all fields for Bus (well, you just make a view)
* Many tables

## Table per concrete class
<img src="images/inheritance4.png" width="40%"/>

### Advantages
* Easy access, all info on an object in one table
* Good space performance

### Disadvantage
Harder to evolve (change to the hierarchy might change many tables)

In [5]:
%%html
<style> 
table td, table th, table tr {text-align:left !important;}
</style>

# Your turn

Which tables should we have, and what sql types to use
<table align="left"><tr>
    <td><img src="images/inheritance5.png" width="60%"/></td>
    
<td align="left">
    <h4>Fields:</h4>
Person: Name string<br>
Person: Emails set.<br>
Student: Enrolled (year) integer<br>
Professor: CV string<br>
TA: hours (how many hours employed) int<br>
Course: Max (no of students) int<br>
    <h4> Relationships:</h4>
enrollment: Many-Many<br>
teaches: Fictively, we assume that a given TA must at most teach one course.<br>
responsible:
A given course has one responsible, who might be responsible for several courses.|
</td></tr></table>


# Semester eval

* One general eval on all courses
* Delphi model 
    - mention 3 things which works well, 
    - and three to be improved
    - and we try to reach concensus
    - and discuss how to solve issue

    
## Delphi in groups of three/four
## Not your working groups
Write your answers here:

https://drive.google.com/open?id=17U6rVbx2DUG747mknIge1sdWzxA6QEwO_q-XMYHM480

# Specifying the mapping


```java
public class Author
{
    private int authorId;
    private String authorName;
    
    private Set<Book> books;
    
    // Omitted constructors and getter/sette
}
```

# XML (old school hipernate)
```xml
<?xml version="1.0"?>
<!DOCTYPE hibernate-mapping PUBLIC "-//Hibernate/Hibernate Mapping DTD 3.0//EN"
"http://hibernate.sourceforge.net/hibernate-mapping-3.0.dtd">
  <hibernate-mapping package="dk.ohno">
    <class name="Author" table="AUTHOR">
      <id name="authorId" column="AUTHOR_ID" type="java.lang.Integer">
        <generator class="native"></generator>
      </id>
      <property name="authorName" column="AUTHOR_NAME"></property>
      <set name="books" inverse="true" lazy="true" cascade="all" fetch="select">
        <key column="AUTHOR_ID" not-null="true" />
           <one-to-many class="Book" />
      </set>
   </class>
</hibernate-mapping>
```

```xml
<set name="books" inverse="true" lazy="true" cascade="all" fetch="select">
    <key column="AUTHOR_ID" not-null="true" />
    <one-to-many class="Book" />
</set>
```
Notice in particular the `books` specification. It specifies:
* `inverse` (foreign key in books), 
* `lazy` load (books are not loaded with authors), 
* `cascade all` (what happens when you delete an author), 
* `fetch=select` (alternative is `join` - and should only be used with `lazy=false`).

Above example was taken from [https://www.javainterviewpoint.com/hibernate-one-many-mapping-example-xml-mapping/](https://www.javainterviewpoint.com/hibernate-one-many-mapping-example-xml-mapping/)

which btw has these three sections:
![](images/xmlmapping1.png)


# Annotation specification
This is the modern way of specifying in .Net and JPA (Java Persistence API)

# JPA Spec
```java
@Entity
@Table(name = "employee")
public class Employee implements Serializable {
   
  @Id
  @Column(name = "id")
  @GeneratedValue
  private int id;
   
  @OneToOne(cascade = CascadeType.MERGE)
  @PrimaryKeyJoinColumn
  private EmployeeDetail employeeDetail;
}
 
@Entity
@Table(name = "employeeDetail")
public class EmployeeDetail implements Serializable {
 
  @Id
  @Column(name = "id")
  private int id;
}
```

# Entity framework Spec
Entity Framework is place emphasis on defaults, but here we override most for clearity

```csharp
[Table("StudentInfo")]
public class Student
{
    public Student() { }
        
    [Key]
    public int SID { get; set; }

    [Column("Name", TypeName="ntext")]
    [MaxLength(20)]
    public string StudentName { get; set; }

    [NotMapped]
    public int? Age { get; set; }
        
        
    public int StdId { get; set; }

    [ForeignKey("StdId")]
    public virtual Standard Standard { get; set; }
}
```

# Loading objects from database

# Mapping a reference (recap)
```java
class Car {
	int KEY;
	string make;
	Person Owner;
}
class Person{
	int Key;
	string name;
}
```
```
Table Car( int: KEY, make: string, Owner: int );
Table Person( int: KEY, name: string);
```

# Loading the Person - when
When do we translate the integer Owner in the Car table to reference Owner in the object?

* When the object is loaded – *eager load*.
* First time the reference field Owner is used – *lazy load*.

The ORM allow you to specify strategy

# Lazy load
Lazy load can be done in different ways:
* A pseudo object/proxy is created. 
* We store the integer version of Owner in the class:

### Proxy
Rather than loading the object, we return an object which implement the same interface, but which have not (yet) retrieved any data from the database.

# Eager load

When you load an object X which has a reference to Y, you also load Y.

Loading both objects requires only one access to the database (you join to get both in one wide table)

### Eager load of collections
Especially important when Y is a collection which can fit in memory.

This prevents lazy loading of each element in the collection.
<img src="images/eagerload.png" width="50%" />

The issue is if you use lazy load you end up with a list of proxy objects, and then accessing each object in the collection will result in a database roundtrip.

# Combinations of eager and lazy are possible:
* Eager load of only some attributes. Eg. lazy load of large pictures
* Eager load of a collection, but only the Name attribute of the elements (to enable showing them in a list)

In general, it is situation dependent what combination is most efficient.

### Choosing the wrong combination can kill you application!


# The table/object manager
To manage the creation of objects, there are at least two important requirements:
* We want to avoid to create the two objects based on the same row. 
* We want make sure we only created objects for those rows which are needed.

A standard way to ensure that only a single copy of each row is created is:
* We notice that each row has a unique key.
* We create a Hashmap, mapping key to objects
* When we ask for an object, we check the map, 
    * if it is *not* there, we get it from the database. 
    * If it is there, we just get a reference to it.

    
# Transactions
A transaction is structured as usual:
```
BEGIN
	read and modify your objects
COMMIT or ABORT
```

One might wonder how to abort a number of changes done to objects. 

## Dirty list

When an object changes its state it is added to a dirty-list. This happens in the setter:
```
class Person {
	…
	private float weight;
	public float Weight{
		get{	… get from database …}
		set{	weight = value;
				ObjectManager.addDirty(this);
}	}	}
```

### Commit or rollback

1. When a transaction begins, the dirty set empty (or it is emptied)

2. No writes to the database occurs during the transaction.

3. A commit can then write all changed objects in the dirty list to the database.

4. A abort will remove all objects in the dirty set from the object manager. They will then be re-read from the database (lazily or eagerly).

# Garbage collection issue
How to prevent the entity manager hashmap from keeping all objects alive forever?

### WeakHashMap

# Queries

Both JPA and .Net has a query language.

# JPA JPQL query language
A well designed query language which works in object terms.
```java
Query query = entitymanager.
    createQuery("Select UPPER(e.ename) from Employee e");
List<String> list = query.getResultList();
```

#### Aggregation
```java
Query query1 = entitymanager.createQuery("Select MAX(e.salary) from Employee e");
Double result = (Double) query1.getSingleResult();
System.out.println("Max Employee Salary :" + result);
```

# LINQ (Language INtegrated Query)
Linq is a query syntax over iterables and queryables.

```csharp
var res = from c in Customers, o in c.Orders, d in o.Details
    where o.quantityOrdered > 10
    select new {c.customerName, o.product.productName}
```

#### Aggregation
```csharp
from c in customers
select new {custName = c.customerName, 
            val = c.orders.Sum(o=> o.orderdetails.Sum(d => d.priceEach))}
```

#### Group by
```csharp
from c in customers
group c by c.country into newGroup
select new {country = newGroup.Key, 
            paym = newGroup.Sum(cust=> cust.payments.Sum(p => p.amount))}
```

# Language integrated
Type safe, compile time check, code completion

Thin layer over iterable:

```csharp
Connection.customers
   .Select (
      c => 
         new  
         {
            custName = c.customerName, 
            val = c.orders.Sum (o => o.orderdetails.Sum (d => d.priceEach))
         }
   )
```

# Linq under the hood

## Translates C# expressions to SQL
Show in Linqpad

## Special methods receive **unevaluated** expressions as arguments
Show in Linqpad the syntax tree of a simple query

## Custom drivers
[LINQ to Mongodb](https://mongodb-documentation.readthedocs.io/en/latest/ecosystem/tutorial/use-linq-queries-with-csharp-driver.html)
```csharp
var query =
    from c in collection.AsQueryable<C>()
    where (c.X % 2 == 0) && (c.X % 3 == 0)
    select c;
```

translates into:
`{ $and : [{ X : { $mod : [2, 0] } }, { X : { $mod : [3, 0] } }] }`

# Your turn to use Linqpad
To be able to use it you have to go through the following setup:

1. A connection to the database must be established. The database we need
is a MySQL database running on digitalocean on a server in frankfurt. To connect to MySQL you must use a 3rd party Linq Provider.
2. On the left side of LinqPad, select "Add connection"
3. You are now prompted to select Data Context. Near the bottom left there is an option called "View more drivers ..." - select it.
4. In my edition, one appears at the top called <br>
"IQ Driver for MySQL, SQLite, Oracle". <br> It works well for me.
5. I'm a little bit uncertain about how it installs (since I've already done it - and "don't rock the boat ..."). But when I select "Add connction", I can now select "IQ" under the list that supports "Build context automatically".
6. You will now be asked about various info to connect to the database. The keywords here are:
  - Server: 46.101.253.149
  - Database: classicmodels
  - User Name: linquser
  - Password: enormslAngt88$kAufmann
  - Port number is set by default to 3306, which is correct.

 This user can only read the database, so there is no major security risk (in addition to any gaps in MySQL).
 
## Your queries

1. for each customer find the total payment done by that customer (print customer name and total payment)
2. same as above, but only for those whose total payment is less than 30000
3. for each productline, find out how many products have been sold. List productline and total
4. find out how many `Vintage Car` products has been sold in each country



# Generating code
![](images/generator1.png)

![](images/generator2.png)

## Different ways to input to the generator
Template based code generation

Web 1.0 model of server side HTML generation

### DSL - Domain specific language
* Often in the form of a json or xml document. (As I did in week 10 on Spatial databases)
* Heavy duty - your own parser and compiler
* Sometimes in the form "dedicated classes"

# Dedicated classes approach
```java
class Attribute {
    String fieldName;
    String typeName;
    public Attribute(String f, String t){ fieldName = f; typeName = t;}
}
class Entity {
    String name;
    List<Attribute> attributes = new ArrayList<>();
    public Entity(String n) {name = n;}
    public Entity add(Attribute a){
        attributes.add(a);
        return this;
    }
}
```

# Sample usage
```java
road = new Entity("Road)
         .add(new Attribute('name','String'))
         .add(new Attribute('maxNo','int'))
         .add(new Attribute('city', 'String'));
```

# Java class generation
```java
class Entity { ... as before
    String asJavaClass(){
        String code = "class " + name + "{\n";
        for (var a : attributes)
            code += a.asJavaCode()+"\n";
        return code+"}";
    }
    String asSQLTableDef(){
        ...
    }
}
```    

# [Assignment](https://github.com/datsoftlyngby/soft2019spring-databases/blob/master/assignments/assignment11.md)

